In [ ]:
!pip install wandb 

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda

from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import wandb

2023-02-18 06:01:58.836252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 06:01:58.939921: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-18 06:01:58.965695: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# PreProcessing

In [2]:
from pathlib import Path

PATH = Path('./CUB_200_2011')
labels = pd.read_csv(PATH/"image_class_labels.txt", header=None, sep=" ")
labels.columns = ["id", "label"]

train_test = pd.read_csv(PATH/"train_test_split.txt", header=None, sep=" ")
train_test.columns = ["id", "is_train"]

images = pd.read_csv(PATH/"images.txt", header=None, sep=" ")
images.columns = ["id", "bird_name"]

classes = pd.read_csv(PATH/"classes.txt", header=None, sep=" ")
classes.columns = ["id", "class"]

df_1 = pd.merge(images, labels, on='id')
df = pd.merge(df_1, train_test, on='id')




train, test = [], []
train_test = df.iloc[:,-1]

for i in range(len(df)):
    if train_test[i] == 0:
        train.append(df.loc[i])
    else:
        test.append(df.loc[i])
        
train = pd.DataFrame(train)
test = pd.DataFrame(test)

print(train.shape)
print(test.shape)

(5794, 4)
(5994, 4)


In [3]:
df

,id,bird_name,label,is_train
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1
...,...,...,...,...
11783,11784,200.Common_Yellowthroat/Common_Yellowthroat_00...,200,1
11784,11785,200.Common_Yellowthroat/Common_Yellowthroat_00...,200,0
11785,11786,200.Common_Yellowthroat/Common_Yellowthroat_00...,200,0
11786,11787,200.Common_Yellowthroat/Common_Yellowthroat_00...,200,1


In [11]:
def preprocessing(image):
    resized_image = tf.image.resize(image, [112,112])
    return tf.keras.applications.densenet.preprocess_input(resized_image)

X_train, y_train = [], []
path = ''

for i, row in train.iterrows():
    bird_name = row['bird_name']
    label = row['label']
    path = f'./CUB_200_2011/images/{bird_name}'
    img = plt.imread(path)
    
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_train.append(img)
        y_train.append(label)
        
X_train = np.array(X_train)
y_train = np.array(y_train)

y_train = tf.keras.utils.to_categorical(y_train)

X_test, y_test = [], []
path = ''

for id, row in test.iterrows():
    bird_name = row['bird_name']
    label = row['label']
    path = f'./CUB_200_2011/images/{bird_name}'
    img = plt.imread(path)
    
    if len(img.shape) == 3 or len(img.shape) == 4:
        img = preprocessing(img)
        X_test.append(img)
        y_test.append(label)
X_test = np.array(X_test)
y_test = np.array(y_test)       
y_test = tf.keras.utils.to_categorical(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5790, 112, 112, 3)
(5790, 201)
(5990, 112, 112, 3)
(5990, 201)


## validation set 생성

In [12]:
from sklearn.model_selection import train_test_split

X_tn, X_val, y_tn, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
print(X_tn.shape) 
print(y_tn.shape) 
print(X_val.shape) 
print(y_val.shape)

(4632, 112, 112, 3)
(4632, 201)
(1158, 112, 112, 3)
(1158, 201)


## Model 생성  _ ResNet152 이용

In [14]:
wandb.init(project="Bird_DenseNet", entity="hcim", name='DenseNet_169_old_pre_1')

categorical_accuracy,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇███████████
loss,██▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_categorical_accuracy,▁▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇██▇▇████▇
val_loss,█▅▃▃▃▂▂▁▁▁▁▁▂▁▁▂▁▁▂▁▂▂▂▂▂▂▂▂▂▂▃▃▂▃▃▃▃▃▃▃
categorical_accuracy,0.98122
loss,0.19931
val_categorical_accuracy,0.13817
val_loss,4.96838


In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D,AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Activation, MaxPool2D, BatchNormalization, Dropout,RandomFlip,RandomRotation

def dense_block(x, blocks, growth_rate):
    for i in range(blocks):
        x = conv_block(x, growth_rate)
    return x

def conv_block(x, growth_rate):
    x1 = BatchNormalization()(x)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(4 * growth_rate, (1, 1), use_bias=False, padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = Activation('relu')(x1)
    x1 = Conv2D(growth_rate, (3, 3), use_bias=False, padding='same')(x1)
    x = tf.keras.layers.Concatenate()([x, x1])
    return x

def transition_block(x, reduction):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), (1, 1), use_bias=False, padding='same')(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x

def densenet(blocks=[6, 12, 32, 32], growth_rate=32, include_top=True, weights=None, input_shape=None, classes=1000):
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(inputs)
    x = Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    for i, num_blocks in enumerate(blocks):
        x = dense_block(x, num_blocks, growth_rate)
        if i != len(blocks) - 1:
            x = transition_block(x, 0.5)

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(classes, activation='softmax')(x)
        
    model = Model(inputs, x, name='densenet')
    return model

model = densenet(input_shape=(112,112,3), classes=201)
model.summary()

Model: "densenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 118, 118, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_168 (Conv2D)            (None, 56, 56, 64)   9408        ['zero_padding2d_2[0][0]']       
                                                                                           

In [16]:
from tensorflow.keras import callbacks
class WandbCallback(callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log(logs)
        
        

from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True, vertical_flip=True, brightness_range=[0.01, 0.3], channel_shift_range=30)
datagen = ImageDataGenerator(rotation_range=15, horizontal_flip=True)
datagen.fit(X_tn)

In [17]:
optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=10e-4)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy()]) 

#hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=64), epochs=150, validation_data=(X_val, y_val), callbacks=[WandbCallback()])
hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=128), epochs=300, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

/tmp/ipykernel_208754/4289859146.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=128), epochs=300, validation_data=(X_val, y_val), callbacks=[WandbCallback()])


Epoch 1/300
37/37 [==============================] - 22s 338ms/step - loss: 5.3927 - categorical_accuracy: 0.0112 - val_loss: 5.5359 - val_categorical_accuracy: 0.0060
Epoch 2/300
37/37 [==============================] - 11s 289ms/step - loss: 5.1940 - categorical_accuracy: 0.0168 - val_loss: 5.4889 - val_categorical_accuracy: 0.0060
Epoch 3/300
37/37 [==============================] - 11s 294ms/step - loss: 4.9165 - categorical_accuracy: 0.0298 - val_loss: 5.2108 - val_categorical_accuracy: 0.0121
Epoch 4/300
37/37 [==============================] - 11s 293ms/step - loss: 4.6203 - categorical_accuracy: 0.0542 - val_loss: 5.0161 - val_categorical_accuracy: 0.0242
Epoch 5/300
37/37 [==============================] - 11s 292ms/step - loss: 4.4211 - categorical_accuracy: 0.0689 - val_loss: 4.9736 - val_categorical_accuracy: 0.0285
Epoch 6/300
37/37 [==============================] - 11s 290ms/step - loss: 4.1882 - categorical_accuracy: 0.0997 - val_loss: 4.8072 - val_categorical_accuracy:

KeyboardInterrupt: 

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wand

In [ ]:
print(model.evaluate(X_train, y_train))
print(model.evaluate(X_val, y_val))

In [ ]:
hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=128), epochs=2700, validation_data=(X_val, y_val), callbacks=[WandbCallback()])

Epoch 1/150


/tmp/ipykernel_17615/2223792544.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(datagen.flow(X_tn, y_tn, batch_size=8), epochs=150, validation_data=(X_val, y_val), callbacks=[early_stopping, WandbCallback()])


652/652 [==============================] - 49s 71ms/step - loss: 0.3170 - accuracy: 0.9060 - f1_score: 0.9007 - val_loss: 4.5993 - val_accuracy: 0.2867 - val_f1_score: 0.2309
Epoch 2/150
652/652 [==============================] - 46s 70ms/step - loss: 0.3048 - accuracy: 0.9096 - f1_score: 0.9040 - val_loss: 4.5025 - val_accuracy: 0.2902 - val_f1_score: 0.2344
Epoch 3/150
652/652 [==============================] - 46s 70ms/step - loss: 0.2869 - accuracy: 0.9146 - f1_score: 0.9101 - val_loss: 4.6246 - val_accuracy: 0.2902 - val_f1_score: 0.2348
Epoch 4/150
652/652 [==============================] - 46s 70ms/step - loss: 0.2665 - accuracy: 0.9188 - f1_score: 0.9152 - val_loss: 4.7421 - val_accuracy: 0.2763 - val_f1_score: 0.2260
Epoch 5/150
652/652 [==============================] - 46s 71ms/step - loss: 0.2440 - accuracy: 0.9284 - f1_score: 0.9229 - val_loss: 4.7851 - val_accuracy: 0.3005 - val_f1_score: 0.2449
Epoch 6/150
652/652 [==============================] - 46s 70ms/step - loss: 

KeyboardInterrupt: 

In [94]:
wandb.finish()

## Model 저장하기

In [36]:
from keras.models import load_model

model.save('bird_Resmodel.h5')